# Convolutional Neural Network with Mnist Dataset

In [1]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import math
from six.moves import xrange
import os.path
import time

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
NUM_CLASSES = 10

# The MNIST images are always 28x28 pixels.
IMAGE_SIZE = 28
#IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
IMAGE_SHAPE = [28, 28, 1]

In [3]:
with tf.name_scope('conv1'):
    a = tf.Variable([1,2], name='biases')
with tf.name_scope('conv2'):
    a = tf.Variable([1,2], name='biases')
    b = tf.Variable([1,2], name='biases')

In [4]:
print(a.name)
print(b.name)

conv2/biases:0
conv2/biases_1:0


In [8]:
tf.reduce_max([2,3,4])

<tf.Tensor 'Max:0' shape=() dtype=int32>

In [15]:
def _inference(images):
    """Build MNIST model
    
    Args:
    images: Images placeholder, from inputs().
    hidden1_units: Size of the first hidden layer.
    hidden2_units: Size of the second hidden layer.
    
    Returns:
    softmax_linear: Output tensor with the computed logits.
    """
    
    def weight_variable(shape, layer):
        initial = tf.truncated_normal(shape,
                                      stddev=1.0 / math.sqrt(float(shape[0])))
        return tf.Variable(initial, name=layer+'/weights')

    def bias_variable(shape, layer):
        #initial = tf.zeros(shape)
        initial = tf.constant(0.1, shape=shape) # prevent dead neural on initialization
        return tf.Variable(initial, name=layer+'/biases')
    
    def conv2d(x, W, layer):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=layer+'/activation') # maintain layer size

    def activation_relu(activation, bias, layer):
        return tf.nn.relu(activation + bias, name =layer+'/activation_relu')
    
    def max_pool_2x2(x, layer):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME', name=layer+'/max_pool') # reduce by 2 layer size
    
    def hist_summary(variable):
        _ = tf.histogram_summary(variable.name, variable)
     
    def scal_summary(variable):
        "Max, min, mean , sstd"
        tf.scalar_summary(variable.name, variable)
        
    # conv layer 1
    with tf.name_scope('conv-maxpool-1'):
        weights = weight_variable(FLAGS.W_conv1, 'conv_1') 
        biases = bias_variable([FLAGS.W_conv1[-1]], 'conv_1')
        
        conv_op = conv2d(images, W_conv1, 'conv_1')
        relu_op = activation_relu(a_conv1, b_conv1, 'conv_1')
        pool_op = max_pool_2x2(h_conv1, 'conv_1')
        
        scal_summary(tf.reduce_max(weights))
        # Add summary ops to collect data
        hist_summary(weight)
        hist_summary(biases)
        hist_summary(conv_op.op)
        hist_summary(relu_op.op)
        hist_summary(pool_op.op)
        

    # conv layer 2
    with tf.name_scope('conv-maxpool-2'):
        W_conv2 = weight_variable(FLAGS.W_conv2, 'conv_2') 
        b_conv2 = bias_variable([FLAGS.W_conv2[-1]], 'conv_2')
        
        a_conv2 = conv2d(h_pool1, W_conv2, 'conv_1')
        h_conv2 = activation_relu(a_conv2, b_conv2, 'conv_2')
        h_pool2 = max_pool_2x2(h_conv2, 'conv_2')
        
        # Add summary ops to collect data
        _ = tf.histogram_summary('conv_2/weights', W_conv2)
        _ = tf.histogram_summary('conv_2/biases', b_conv2)
        _ = tf.histogram_summary('conv_2/activation', h_conv2)        
    
    # fc layer 1
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable(FLAGS.W_fc1, 'W_fc1') 
        b_fc1 = bias_variable([FLAGS.W_fc1[-1]], 'b_fc1')
        h_pool2_flat = tf.reshape(h_pool2, [-1, FLAGS.W_fc1[0]])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        
        # Add summary ops to collect data
        _ = tf.histogram_summary('fc_1/W_fc1', W_fc1)
        _ = tf.histogram_summary('fc_1/b_fc1', b_fc1)
        _ = tf.histogram_summary('fc_1/h_fc1', h_fc1)
        
    # dropout layer
    with tf.name_scope('dropout'):
        h_fc1_drop = tf.nn.dropout(h_fc1, FLAGS.dropout)
        
    # Linear softmax layer
    with tf.name_scope('softmax_linear'):
        W_fc2 = weight_variable(FLAGS.W_fc2, 'W_fc2') 
        b_fc2 = bias_variable([FLAGS.W_fc2[-1]], 'b_fc2')
        y_logit = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
        
        # Add summary ops to collect data
        _ = tf.histogram_summary('fc_2/W_fc2', W_fc2)
        _ = tf.histogram_summary('fc_2/b_fc2', b_fc2)
        _ = tf.histogram_summary('fc_2/y_logit', y_logit)     

    
    return y_logit

In [4]:
def _loss(logits, labels):
    """Calculates the loss.
    
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size].
    
    Returns:
    loss: Loss tensor of type float.
    """
    # Convert from sparse integer labels in the range [0, NUM_CLASSES)
    # to 1-hot dense float vectors.
    
    batch_size = tf.size(labels)
    labels = tf.expand_dims(labels, 1)
    indices = tf.expand_dims(tf.range(0, batch_size), 1)
    concated = tf.concat(1, [indices, labels])
    
    onehot_labels = tf.sparse_to_dense(concated,
                                       tf.pack([batch_size, NUM_CLASSES]),
                                       1.0, 0.0)
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits,
                                                            onehot_labels,
                                                            name='xentropy')
    
  
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    #_ = tf.scalar_summary('cross entropy', loss)
    return loss

In [5]:
def _training(loss, learning_rate):
    """Sets up the training Ops.
    
    Args:
    loss: Loss tensor.
    learning_rate: The learning rate to use for gradient descent.
    
    Returns:
    train_op: The Op for training.
    """
    # summarizer to track the loss over time in TensorBoard.
    tf.scalar_summary(loss.op.name, loss)
    
    # gradient descent optimizer
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, FLAGS.momentum)
    
    # track global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)
    
    # single training step.
    #train_op = optimizer.minimize(loss, global_step=global_step)
    compute_gradients = optimizer.compute_gradients(loss)
    for i, (grad, _) in enumerate(compute_gradients):
        _ = tf.histogram_summary('gradients/grad-%s' %i, grad)
        
    train_op = optimizer.apply_gradients(compute_gradients)  
    return train_op

In [6]:
def _evaluation(logits, labels):
    """Evaluate the quality of the predictions.
    
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
      
    Returns:
    A scalar int32 tensor with the number of examples
    that were predicted correctly.
    """
    # returns a bool tensor with shape [batch_size]
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.

    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    _ = tf.scalar_summary('accuracy', accuracy)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [7]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    
    Args:
    batch_size: The batch size.
    
    Returns:
    images_placeholder: Images placeholder.
    labels_placeholder: Labels placeholder.
    """
    images_placeholder = tf.placeholder(tf.float32, shape=[batch_size] + IMAGE_SHAPE, name='image-input')
    labels_placeholder = tf.placeholder(tf.int32, shape=[batch_size], name='target-label')
    return images_placeholder, labels_placeholder

In [8]:
def fill_feed_dict(data_set, images_pl, labels_pl):
    """Fills the feed_dict for training the given step.

    Args:
        data_set: The set of images and labels, from input_data.read_data_sets()
        images_pl: The images placeholder, from placeholder_inputs().
        labels_pl: The labels placeholder, from placeholder_inputs().
        
    Returns:
        feed_dict: The feed dictionary mapping from placeholders to values.
      """
    images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size,
                                                   FLAGS.fake_data)
    
    x_image = np.reshape(images_feed, [FLAGS.batch_size] + IMAGE_SHAPE)
    
    feed_dict = {
        images_pl: x_image,
        labels_pl: labels_feed,
        }
    return feed_dict

In [9]:
def do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_set):
    """Runs one evaluation against the full epoch of data.
    
    Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
    """
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = data_set.num_examples // FLAGS.batch_size
    num_examples = steps_per_epoch * FLAGS.batch_size
    
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set,
                                   images_placeholder,
                                   labels_placeholder)
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = true_count / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))

In [10]:
def run_training():
    """Train MNIST for a number of steps."""
    # Get the sets of images and labels for training, validation, and
    # test on MNIST.
    data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
        
        with tf.name_scope('input-data'):
            # Generate placeholders for the images and labels.
            images_placeholder, labels_placeholder = placeholder_inputs(FLAGS.batch_size)

        # Build a Graph that computes predictions from the inference model.
        logits = _inference(images_placeholder)

        with tf.name_scope('xentropy_loss'):
            # Add to the Graph the Ops for loss calculation.
            loss = _loss(logits, labels_placeholder)

        with tf.name_scope('train'):
            # Add to the Graph the Ops that calculate and apply gradients.
            train_op = _training(loss, FLAGS.learning_rate)

        with tf.name_scope('evaluation'):
            # Add the Op to compare the logits to the labels during evaluation.
            eval_correct = _evaluation(logits, labels_placeholder)

        # Build the summary operation based on the TF collection of Summaries.
        summary_op = tf.merge_all_summaries()

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()

        # Create a session for running Ops on the Graph.
        sess = tf.Session()

        # Run the Op to initialize the variables.
        init = tf.initialize_all_variables()
        sess.run(init)

        # Instantiate a SummaryWriter to output summaries and the Graph.
        summary_writer = tf.train.SummaryWriter('logs/',
                                                graph_def=sess.graph_def)

        # And then after everything is built, start the training loop.
        for step in xrange(FLAGS.max_steps):
            start_time = time.time()

            # Fill a feed dictionary with the actual set of images and labels
            # for this particular training step.
            feed_dict = fill_feed_dict(data_sets.train,
                                       images_placeholder,
                                       labels_placeholder)

            # Run one step of the model.  The return values are the activations
            # from the `train_op` (which is discarded) and the `loss` Op.  To
            # inspect the values of your Ops or variables, you may include them
            # in the list passed to sess.run() and the value tensors will be
            # returned in the tuple from the call.
            _, loss_value = sess.run([train_op, loss],
                                     feed_dict=feed_dict)

            duration = time.time() - start_time

            # Write the summaries and print an overview fairly often.
            if step % 100 == 0:
                # Print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                # Update the events file.
                summary_str = sess.run(summary_op, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str, step)

            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % 1000 == 0 or (step + 1) == FLAGS.max_steps:
                saver.save(sess, FLAGS.train_dir, global_step=step)
                # Evaluate against the training set.
                print('Training Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        data_sets.train)
                # Evaluate against the validation set.
                print('Validation Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        data_sets.validation)
                # Evaluate against the test set.
                print('Test Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        data_sets.test)

In [11]:
# Basic model parameters as external flags.
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_float('momentum', 0.8, 'momentum hyperparameters')
flags.DEFINE_integer('max_steps', 200, 'Number of steps to run trainer.')
flags.DEFINE_integer('batch_size', 128, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
                     'for unit testing.')
flags.DEFINE_string('log_dir', 'logs', 'Directory to put the log data.')
flags.DEFINE_float('dropout', 0.5, 'dropout ratio')
flags.DEFINE_integer('W_conv1', [5,5,1,32], 'W shape of 1st conv layer')
flags.DEFINE_integer('W_conv2', [5,5,32,64], 'W shape of 2st conv layer')
flags.DEFINE_integer('W_fc1', [7*7*64, 1024], 'W shape of 1st fc layer') # 28/2/2 = 7 after 2 pooling layers
flags.DEFINE_integer('W_fc2', [1024, 10], 'W shape of 2st fc layer') # 10 output classes

In [ ]:
# Momentum updater => momentum: 0.8, dropout: 0.5, lr: 0.001, batch: 128
run_training()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Step 0: loss = 8.07 (0.555 sec)


In [ ]:
# Gradient Descent => dropout: 0.8, lr: 0.01, batch: 100
run_training() 

## Manual Implementation of Updaters

In [8]:
def Momentum(loss, learning_rate, mu):
    dy = tf.gradients(loss, W)
    grad = dy[0]

    vW_op = tf.assign(vW, (mu * vW) - (learning_rate * grad))
    train_op = tf.assign_add(W, vW)
    return train_op, vW_op

In [11]:
def Nesterov(loss, learning_rate, mu):
    dy = tf.gradients(loss, W)
    grad = dy[0]
    
    vW_prev = vW
    vW_op = tf.assign(vW, (mu * vW) - (learning_rate * grad))
    train_op = tf.assign_add(W, -mu * vW_prev + (1+mu) * vW)
    return train_op, vW_op

In [25]:
def Adadelta(loss, decay):
    dy = tf.gradients(loss, W)
    grad = dy[0]
    
    vW_op = tf.assign(vW, decay * vW + (1-decay)* tf.pow(grad,2))
    W_update = - tf.sqrt(mW + 1e-8) * grad / tf.sqrt(vW + 1e-8)
    
    mW_op = tf.assign(mW, decay * mW + (1-decay)* tf.pow(W_update,2))
    
    train_op = tf.assign_add(W, W_update)
    return train_op, vW_op, mW_op

In [31]:
def Adam(loss, learning_rate, beta1, beta2):
    dy = tf.gradients(loss, W)
    grad = dy[0]
    
    mW_op = tf.assign(mW, beta1*mW + (1-beta1) * grad)
    vW_op = tf.assign(vW, beta2*vW + (1-beta2) * tf.pow(grad,2))
    
    mW_prime = mW / (1-beta1)
    vW_prime = vW / (1-beta2)
    
    train_op = tf.assign_add(W, - learning_rate * mW_prime / tf.sqrt(vW_prime + 1e-8))
    return train_op, vW_op, mW_op             

## Test Example

In [37]:
# Construct Graph input nodes
vector = np.array([0.0, 0.0, 0.0])
target = np.array([1.0, 2.0, 3.0])
learning_rate = 0.5
mu = 0.5
beta1 = 0.2
beta2 = 0.2
#optimizer = "Momentum"
optimizer = "Adadelta"
#optimizer = "Adam"


init_vect = tf.constant(vector, tf.float32, vector.shape)
W = tf.Variable(init_vect, name='W')
vW = tf.Variable(init_vect, name='vW')
mW = tf.Variable(init_vect, name='mW')

y = tf.placeholder(tf.float32, shape=target.shape, name='target')

# Construct Graph operations
err = W - y
cost = - tf.exp((-0.5) * tf.reduce_sum(tf.pow(err,2)))

if optimizer == "Adadelta":
    train_op, vW_op, mW_op = Adadelta(cost, mu)
elif optimizer == "Adam":
    train_op, vW_op, mW_op = Adam(cost, learning_rate, beta1, beta2)
else:
    train_op, vW_op, = Momentum(cost, learning_rate, mu)

# Start a session
sess = tf.Session()

# Run the Op to initialize the variables.
init = tf.initialize_all_variables()
sess.run(init)

for i in xrange(200):
    
    if optimizer == "Adadelta" or optimizer == "Adam":
        _,_,_,loss_value = sess.run([train_op, vW_op, mW_op, cost], feed_dict={y: target})
    else:
         _,_,loss_value = sess.run([train_op, vW_op, cost], feed_dict={y: target})

    if i % 20 == 0:
        print (i, ": ", W.eval(session=sess))
        print ("Cost: ", loss_value)
        print()

0 :  [ 0.00091188  0.00182376  0.00273565]
Cost:  -0.000911882

20 :  [ 0.02182399  0.04364799  0.06547198]
Cost:  -0.0012137

40 :  [ 0.05026133  0.10052267  0.15078403]
Cost:  -0.00177059

60 :  [ 0.09390396  0.18780796  0.281712  ]
Cost:  -0.00308056

80 :  [ 0.1830676   0.36613521  0.54920298]
Cost:  -0.00862341

100 :  [ 0.99999368  1.99998736  2.99998093]
Cost:  -0.999345

120 :  [ 1.  2.  3.]
Cost:  -1.0

140 :  [ 1.  2.  3.]
Cost:  -1.0

160 :  [ 1.  2.  3.]
Cost:  -1.0

180 :  [ 1.  2.  3.]
Cost:  -1.0



In [21]:
for i, (a,b) in enumerate([(1,2),(2,3), (3,4)]):
    print(i)
    print(a)
    print (b)

0
1
2
1
2
3
2
3
4
